#**ACCELERATED IMAGE RECOMMENDATION SYSTEM**

### **PART 1: PARALLELIZING IMAGE RECOMMENDATION SYSTEM** 

# Import Statements

In [1]:
import os
import cv2
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed

# Loading Features of Database Images and Query Image extracted from:
1. RGB3D Histogram and Hu Moments
2. MobileNet Features

In [2]:
queryVector_rgb3d = np.load('/content/blue359_rgb.npy')
queryVector_mobilenet = np.load('/content/drive/MyDrive/blue359_mobilenet.npy')

features = np.load('/content/drive/MyDrive/image_features.npy',allow_pickle=True) 
features_flat = features.flat[0]
features_rgb3d = list(features_flat.items())  

features = np.load('/content/drive/MyDrive/mobilenet_features.npy',allow_pickle=True)   
features_flat = features.flat[0]
features_mobilenet = list(features_flat.items())

# Distance Measures

## 1. Euclidean Distance

In [7]:
def euclidean_distance(queryVector, feature): 
    img = feature[0]
    vec = feature[1]
    dist = np.sum((vec[:,None,:] - queryVector[None,:,:])**2, axis=2)**0.5
    return img,dist

## 2. Manhattan Distance

In [8]:
def manhattan_distance(queryVector, feature): 
    img = feature[0]
    vec = feature[1]
    temp = np.sum((queryVector - vec))
    dist = np.abs(temp)
       
    return img,dist

## 3. Chi Squared Distance

In [9]:
def chi_squared(queryVector, feature):
    img = feature[0]
    vec = feature[1]

    num = ((vec[:,None,:] - queryVector[None,:,:])**2)
    den = (vec[:,None,:] + queryVector[None,:,:])
    dist = np.sum(num/den)
       
    return img,dist

## 4. Cosine Similarity 

In [10]:
def cosine(queryVector, feature): 
    img = feature[0]
    vec = feature[1]


    dot_product = np.dot(queryVector, np.transpose(vec))
    norm_1 = np.linalg.norm(queryVector)  
    norm_2 = np.linalg.norm(vec)
    dist =  dot_product / (norm_1 * norm_2)
    dist = 1- dist
    return img,dist

# Baseline Model - Serial Implementation of Feature Comparison

In [12]:
def serial_calculate(queryVector,features,function):
    measures = []
    start = time.perf_counter()
    for f in features:
        imagename, val = function(queryVector,f)
        measures.append((imagename, val))
    end = time.perf_counter()
    ser_time = {end-start}
    print("The serial time taken for",function,ser_time)
    return measures

## Serial Time Calculation for Image features extracted using RGB3D histogram and Hu Moements

In [ ]:
measures_euclidean = serial_calculate(queryVector_rgb3d,features_rgb3d,euclidean_distance)

measures_manhattan = serial_calculate(queryVector_rgb3d,features_rgb3d,manhattan_distance)

measures_chi_squared = serial_calculate(queryVector_rgb3d,features_rgb3d,chi_squared)

measures_cosine = serial_calculate(queryVector_rgb3d,features_rgb3d,cosine)

## Displaying 10 recommended products using Cosine Similarity as an example

In [ ]:
measures_cosine.sort(key=lambda res: res[1])

def getImg(img):
    image_db_path = "/content/PDC_dataset/"
    image = cv2.imread(image_db_path+img)
    image = cv2.resize(image, (500,500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

plt.figure(figsize=(20,20)) 

for i in range(10):
    plt.subplot(5,5,i+1)
    plt.imshow(getImg(measures_cosine[i][0]))

plt.show()

## Serial Time Calculation for Image features extracted using MobileNet

In [13]:
measures_euclidean = serial_calculate(queryVector_mobilenet,features_mobilenet,euclidean_distance)

measures_manhattan = serial_calculate(queryVector_mobilenet,features_mobilenet,manhattan_distance)

measures_chi_squared = serial_calculate(queryVector_mobilenet,features_mobilenet,chi_squared)

measures_cosine = serial_calculate(queryVector_mobilenet,features_mobilenet,cosine)

The serial time taken for <function euclidean_distance at 0x7fbacef66e50> {0.6575117859993043}
The serial time taken for <function manhattan_distance at 0x7fbacef66f70> {0.5567579929993371}
The serial time taken for <function chi_squared at 0x7fbacef66c10> {0.81204229100058}


<ipython-input-10-0e9ee634f691>:9: RuntimeWarning: invalid value encountered in true_divide
  dist =  dot_product / (norm_1 * norm_2)


The serial time taken for <function cosine at 0x7fbacef66040> {1.1349798749997717}


## Displaying 10 recommended products using Chi Squared Similarity as an example

In [ ]:
measures_chi_squared.sort(key=lambda res: res[1])

def getImg(img):
    image_db_path = "/content/PDC_dataset/"
    image = cv2.imread(image_db_path+img)
    image = cv2.resize(image, (500,500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

plt.figure(figsize=(20,20)) 

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(getImg(measures_cosine[i][0]))

plt.show()

## Parallel Implementation of Feature Comparison using Multi Threading 
  

*   For thread no = 2,4,8,16,32 and 64











In [19]:
def parallel_calculate(queryVector,features,threads,function):
  start = time.perf_counter()
  with ThreadPoolExecutor(threads) as executor:
    futures = [executor.submit(function, queryVector , f) for f in features]

  for future in as_completed(futures):
    res = future.result()

  end = time.perf_counter()
  print(f"Parallel Time for :",function, {end-start})

## Parallel Time Calculation for Image features extracted using RGB3D histogram and Hu Moements

In [ ]:
measures_euclidean = parallel_calculate(queryVector_rgb3d,features_rgb3d,64,euclidean_distance)

measures_manhattan = parallel_calculate(queryVector_rgb3d,features_rgb3d,64,manhattan_distance)

measures_chi_squared = parallel_calculate(queryVector_rgb3d,features_rgb3d,64,chi_squared)

measures_cosine = parallel_calculate(queryVector_rgb3d,features_rgb3d,64,cosine)

### Displaying 10 recommended products using Cosine Similarity as an example

In [ ]:
measures_cosine.sort(key=lambda res: res[1])

def getImg(img):
    image_db_path = "/content/PDC_dataset/"
    image = cv2.imread(image_db_path+img)
    image = cv2.resize(image, (500,500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

plt.figure(figsize=(20,20)) 

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(getImg(measures_cosine[i][0]))

plt.show()

## Parallel Time Calculation for Image features extracted using MobileNet

In [20]:
measures_euclidean = parallel_calculate(queryVector_mobilenet,features_mobilenet,64,euclidean_distance)

measures_manhattan = parallel_calculate(queryVector_mobilenet,features_mobilenet,64,manhattan_distance)

measures_chi_squared = parallel_calculate(queryVector_mobilenet,features_mobilenet,64,chi_squared)

measures_cosine = parallel_calculate(queryVector_mobilenet,features_mobilenet,64,cosine)

Parallel Time for : <function euclidean_distance at 0x7fbacef66e50> {5.729642222999246}
Parallel Time for : <function manhattan_distance at 0x7fbacef66f70> {4.863444096999956}
Parallel Time for : <function chi_squared at 0x7fbacef66c10> {6.5278056469996955}


<ipython-input-10-0e9ee634f691>:9: RuntimeWarning: invalid value encountered in true_divide
  dist =  dot_product / (norm_1 * norm_2)


Parallel Time for : <function cosine at 0x7fbacef66040> {6.241153443999792}


### Displaying 10 recommended products using Chi Squared Similarity as an example

In [ ]:
measures_chi_squared.sort(key=lambda res: res[1])

def getImg(img):
    image_db_path = "/content/PDC_dataset/"
    image = cv2.imread(image_db_path+img)
    image = cv2.resize(image, (500,500))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

plt.figure(figsize=(20,20)) 

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(getImg(measures_cosine[i][0]))

plt.show()